# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [63]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from newsapi import NewsApiClient
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
load_dotenv()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\15613\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [64]:
# Read your api key environment variable
api_key = os.getenv("NEWS_API_KEY")

In [65]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [89]:
# Fetch the Bitcoin news articles
bitcoin_headlines = newsapi.get_everything(q="bitcoin",
                                          language="en",
                                          sort_by="relevancy")
bitcoin_headlines


{'status': 'ok',
 'totalResults': 9477,
 'articles': [{'source': {'id': 'wired', 'name': 'Wired'},
   'author': 'Arielle Pardes',
   'title': 'Miami’s Bitcoin Conference Left a Trail of Harassment',
   'description': 'For some women, inappropriate conduct from other conference-goers continued to haunt them online.',
   'url': 'https://www.wired.com/story/bitcoin-2022-conference-harassment/',
   'urlToImage': 'https://media.wired.com/photos/627a89e3e37e715cb7d760d2/191:100/w_1280,c_limit/Bitcoin_Miami_Biz_GettyImages-1239817123.jpg',
   'publishedAt': '2022-05-10T16:59:46Z',
   'content': 'Now, even though there are a number of women-focused crypto spaces, Odeniran says women are still underrepresented. Ive been in spaces where Im the only Black person, or the only woman, or the only B… [+3828 chars]'},
  {'source': {'id': 'the-verge', 'name': 'The Verge'},
   'author': 'Justine Calma',
   'title': 'Why fossil fuel companies see green in Bitcoin mining projects',
   'description': 'Exxo

In [90]:
# Fetch the Ethereum news articles
ethereum_headlines = newsapi.get_everything(q='Ethereum',
                                          language='en',
                                          sort_by="relevancy")
ethereum_headlines

{'status': 'ok',
 'totalResults': 4549,
 'articles': [{'source': {'id': 'engadget', 'name': 'Engadget'},
   'author': 'Jon Fingas',
   'title': "Here's what NFTs look like on Instagram",
   'description': "Meta has revealed more of how NFTs will work on Instagram. In the US-based test, you can show what you've bought or created for free by connecting your Instagram account to a compatible digital wallet and posting for the world to see. If you like, the social …",
   'url': 'https://www.engadget.com/instagram-nft-details-131020868.html',
   'urlToImage': 'https://s.yimg.com/os/creatr-uploaded-images/2022-05/2546c160-d05e-11ec-b75e-e45eaa8c5b2b',
   'publishedAt': '2022-05-10T13:10:20Z',
   'content': "Meta has revealed more of how NFTs will work on Instagram. In the US-based test, you can show what you've bought or created for free by connecting your Instagram account to a compatible digital walle… [+1223 chars]"},
  {'source': {'id': None, 'name': 'Gizmodo.com'},
   'author': 'Kyle Ba

In [91]:
# Create the Bitcoin sentiment scores DataFrame

bitcoin_sentiments = []

for article in bitcoin_headlines["articles"]:
    try:
        text = article["content"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        bitcoin_sentiments.append({
            "text": text,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
bitcoin_df = pd.DataFrame(bitcoin_sentiments)

# Reorder DataFrame columns
cols = ["compound", "negative", "neutral", "positive", "text"]
bitcoin_df = bitcoin_df[cols]

bitcoin_df.head()

,compound,negative,neutral,positive,text
0,0.0772,0.000,0.964,0.036,"Now, even though there are a number of women-f..."
1,-0.0516,0.061,0.882,0.056,A Bitcoin mining site powered by otherwise los...
2,-0.3269,0.143,0.772,0.085,Warren Buffett has always been a bitcoin skept...
3,0.3818,0.052,0.833,0.114,"As a kid, I remember when my father tried to u..."
4,0.3400,0.000,0.928,0.072,"Image source, Getty Images\r\nThe value of Bit..."


In [92]:
# Create the Ethereum sentiment scores DataFrame
ethereum_sentiments = []

for article in ethereum_headlines["articles"]:
    try:
        text = article["content"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        ethereum_sentiments.append({
            "text": text,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
ethereum_df = pd.DataFrame(ethereum_sentiments)

# Reorder DataFrame columns
cols = ["compound", "negative", "neutral", "positive", "text"]
ethereum_df = ethereum_df[cols]

ethereum_df.head()

,compound,negative,neutral,positive,text
0,0.6486,0.000,0.865,0.135,Meta has revealed more of how NFTs will work o...
1,-0.1027,0.040,0.960,0.000,GameStop has officially thrown itself headlong...
2,-0.2263,0.075,0.879,0.046,A multi-billion dollar cryptocurrency company ...
3,-0.2732,0.055,0.945,0.000,When Bored Ape Yacht Club creators Yuga Labs a...
4,0.1280,0.000,0.954,0.046,GameStop is going all-in on crypto. The video ...


In [93]:
# Describe the Bitcoin Sentiment
bitcoin_df.describe()

,compound,negative,neutral,positive
count,20.000000,20.000000,20.000000,20.000000
mean,-0.119220,0.084900,0.851800,0.063350
std,0.404397,0.075734,0.100743,0.060985
min,-0.859300,0.000000,0.557000,0.000000
25%,-0.386825,0.057250,0.827000,0.000000
50%,-0.284600,0.071500,0.878000,0.055000
75%,0.156025,0.103750,0.923000,0.085000
max,0.750600,0.300000,0.964000,0.202000


In [94]:
# Describe the Ethereum Sentiment
ethereum_df.describe()

,compound,negative,neutral,positive
count,20.000000,20.000000,20.000000,20.000000
mean,-0.088800,0.048600,0.918700,0.032800
std,0.343530,0.043472,0.051321,0.051042
min,-0.690800,0.000000,0.822000,0.000000
25%,-0.284450,0.000000,0.875500,0.000000
50%,-0.153100,0.050000,0.935000,0.000000
75%,0.045025,0.069000,0.955000,0.051750
max,0.690800,0.178000,1.000000,0.178000


### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [95]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [96]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
sw_addon = {}
sw = sw.union(sw_addon)

In [97]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
   
    # Create a tokenized list of the words
    words = word_tokenize(re_clean)
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    # Remove the stop words
    tokens = [word.lower() for word in lem if word.lower() not in sw]
    
    return tokens

In [105]:
test_df = tokenizer(bitcoin_df['text'][2])
test_df

['warren',
 'buffett',
 'ha',
 'always',
 'bitcoin',
 'skeptic',
 'billionaire',
 'investor',
 'harsh',
 'word',
 'yet',
 'cryptocurrency',
 'weekend',
 'buffett',
 'made',
 'clear',
 'opposition',
 'crypt',
 'char']

In [110]:
# Create a new tokens column for Bitcoin
for index, row in bitcoin_df.iterrows():
    try:
       bitcoin_df['tokens'][row] = tokenizer(bitcoin_df['text'][index])
    except AttributeError:
        pass
bitcoin_df.head()

KeyError: 'tokens'

In [14]:
# Create a new tokens column for Ethereum
for index, row in bitcoin_df.iterrows():
    try:
       ethereum_df['tokens'].append(tokenizer(ethereum_df['text']))
    except AttributeError:
        pass
ethereum_df.head()

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [111]:
from collections import Counter
from nltk import ngrams

In [112]:
# Generate the Bitcoin N-grams where N=2
bitcoin_bigram_counts = Counter(ngrams(bitcoin_df['text'],n=2))
print(dict(bitcoin_bigram_counts))

{('Now, even though there are a number of women-focused crypto spaces, Odeniran says women are still underrepresented. Ive been in spaces where Im the only Black person, or the only woman, or the only B… [+3828 chars]', 'A Bitcoin mining site powered by otherwise lost gas from an oil well near Linden, Texas, on April 4th, 2022. | Photo by FRANCOIS PICARD/AFP via Getty Images\r\n\n \n\n Of all the corporate climate hype fl… [+6544 chars]'): 1, ('A Bitcoin mining site powered by otherwise lost gas from an oil well near Linden, Texas, on April 4th, 2022. | Photo by FRANCOIS PICARD/AFP via Getty Images\r\n\n \n\n Of all the corporate climate hype fl… [+6544 chars]', 'Warren Buffett has always been a bitcoin skeptic. But the billionaire investor had his most harsh words yet for the cryptocurrency over the weekend. Buffett made it clear that his opposition to crypt… [+2297 chars]'): 1, ('Warren Buffett has always been a bitcoin skeptic. But the billionaire investor had his most harsh words y

In [113]:
# Generate the Ethereum N-grams where N=2
ethereum_bigram_counts = Counter(ngrams(ethereum_df['text'],n=2))
print(dict(ethereum_bigram_counts))

{("Meta has revealed more of how NFTs will work on Instagram. In the US-based test, you can show what you've bought or created for free by connecting your Instagram account to a compatible digital walle… [+1223 chars]", 'GameStop has officially thrown itself headlong into the web3 vipers nest with a new app release, though its hard to say whether its proposed population of gamers and game developers will take up the … [+3255 chars]'): 1, ('GameStop has officially thrown itself headlong into the web3 vipers nest with a new app release, though its hard to say whether its proposed population of gamers and game developers will take up the … [+3255 chars]', 'A multi-billion dollar cryptocurrency company has apologised to users after its sale of metaverse land sparked a frenzy that temporarily brought down the Ethereum cryptocurrency.\r\nYuga Labs, the comp… [+3475 chars]'): 1, ('A multi-billion dollar cryptocurrency company has apologised to users after its sale of metaverse land sparked a 

In [114]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [116]:
# Use token_count to get the top 10 words for Bitcoin
bitcoin_top_10 = token_count(bitcoin_bigram_counts)
bitcoin_top_10

[(('Now, even though there are a number of women-focused crypto spaces, Odeniran says women are still underrepresented. Ive been in spaces where Im the only Black person, or the only woman, or the only B… [+3828 chars]',
   'A Bitcoin mining site powered by otherwise lost gas from an oil well near Linden, Texas, on April 4th, 2022. | Photo by FRANCOIS PICARD/AFP via Getty Images\r\n\n \n\n Of all the corporate climate hype fl… [+6544 chars]'),
  1),
 (('A Bitcoin mining site powered by otherwise lost gas from an oil well near Linden, Texas, on April 4th, 2022. | Photo by FRANCOIS PICARD/AFP via Getty Images\r\n\n \n\n Of all the corporate climate hype fl… [+6544 chars]',
   'Warren Buffett has always been a bitcoin skeptic. But the billionaire investor had his most harsh words yet for the cryptocurrency over the weekend. Buffett made it clear that his opposition to crypt… [+2297 chars]'),
  1),
 (('Warren Buffett has always been a bitcoin skeptic. But the billionaire investor had his m

In [117]:
# Use token_count to get the top 10 words for Ethereum
ethereum_top_10 = token_count(ethereum_bigram_counts)
ethereum_top_10

[(("Meta has revealed more of how NFTs will work on Instagram. In the US-based test, you can show what you've bought or created for free by connecting your Instagram account to a compatible digital walle… [+1223 chars]",
   'GameStop has officially thrown itself headlong into the web3 vipers nest with a new app release, though its hard to say whether its proposed population of gamers and game developers will take up the … [+3255 chars]'),
  1),
 (('GameStop has officially thrown itself headlong into the web3 vipers nest with a new app release, though its hard to say whether its proposed population of gamers and game developers will take up the … [+3255 chars]',
   'A multi-billion dollar cryptocurrency company has apologised to users after its sale of metaverse land sparked a frenzy that temporarily brought down the Ethereum cryptocurrency.\r\nYuga Labs, the comp… [+3475 chars]'),
  1),
 (('A multi-billion dollar cryptocurrency company has apologised to users after its sale of metavers

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [118]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [119]:
# Generate the Bitcoin word cloud
bitcoin_wc = WordCloud().generate(bitcoin_df['tokens'])
plt.imshow(bitcoin_wc, title="Bitcoin Word Cloud")

KeyError: 'tokens'

In [120]:
# Generate the Ethereum word cloud
ethereum_wc = WordCloud().generate(ethereum_df['tokens'])
plt.imshow(ethereum_wc, title="Ethereum Word Cloud")

KeyError: 'tokens'

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [121]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [122]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [123]:
# Concatenate all of the Bitcoin text together
bitcoin_corpus = ""
for index, row in bitcoin_df.iterrows():
    bitcoin_corpus += bitcoin_df['text']

bitcoin_string = ' '.join(bitcoin_corpus)


In [124]:
# Run the NER processor on all of the text
bitcoin_doc = nlp(bitcoin_string)

# Add a title to the document
title = "Bitcoin NER"

In [126]:
# Render the visualization
displacy.render(bitcoin_doc, style='ent')

In [127]:
# List all Entities
for ent in bitcoin_doc.ents:
    print(ent.text, ent.label_)


Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Odeniran GPE
Linden GPE
Texas GPE
April 4th, 2022 DATE
Photo PERSON
FRANCOIS PICARD/AFP ORG
Getty Images ORG
Linden GPE
Texas GPE
April 4th, 2022 DATE
Photo PERSON
FRANCOIS PICARD/AFP ORG
Getty Images ORG
Linden GPE
Texas GPE
April 4th, 2022 DATE
Photo PERSON
FRANCOIS PICARD/AFP ORG
Getty Images ORG
Linden GPE
Texas GPE
April 4th, 2022 DATE
Photo PERSON
FRANCOIS PICARD/AFP ORG
Getty Images ORG
Linden GPE
Texas GPE
April 4th, 2022 DATE
Photo PERSON
FRANCOIS PICARD/AFP ORG
Getty Images ORG
Linden GPE
Texas GPE
April 4th, 2022 DATE
Photo PERSON
FRANCOIS PICARD/AFP ORG
Getty Images ORG
Linden GPE
Texas GPE
April 4th, 2022 DATE
Photo PERSON
FRANCOIS PICARD/AFP ORG
Getty Images ORG
Linden GPE
Texas GPE
April 4th, 2022 DATE
Photo PERSON
FRANCOIS PICA

---

### Ethereum NER

In [128]:
# Concatenate all of the Ethereum text together
ethereum_corpus = ""
for index, row in ethereum_df.iterrows():
    ethereum_corpus += ethereum_df['text']

ethereum_string = ' '.join(ethereum_corpus)


In [129]:
# Run the NER processor on all of the text
ethereum_doc = nlp(ethereum_string)

# Add a title to the document
title = "Ethereum NER"

In [130]:
# Render the visualization
displacy.render(ethereum_doc, style='ent')

In [131]:
# List all Entities
for ent in ethereum_doc.ents:
    print(ent.text, ent.label_)


Meta ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
Instagram ORG
US GPE
Instagram ORG
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
web3 LOC
Ethereum ORG
Yuga Labs PERSON
Ethereum ORG
Yuga Labs PERSON
Ethereum ORG
Yuga Labs PERSON
Ethereum ORG
Yuga Lab

---